# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
# your code here

db = pd.read_csv('C:\\Users\\User\\Desktop\\IronHack\\LABS\\lab-advanced-pandas\\your-code\\data\\Admission_Predict.csv')

db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Remove trailing spaces at the end of the column names if there are any.


In [3]:
# your code here
db.columns=db.columns.str.replace(' ','')


Let's evaluate the dataset by looking at the `head` function.

In [4]:
# your code here
db.head()

,SerialNo.,GREScore,TOEFLScore,UniversityRating,SOP,LOR,CGPA,Research,ChanceofAdmit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
# your code here
db.isnull().sum()

SerialNo.           0
GREScore            0
TOEFLScore          0
UniversityRating    0
SOP                 0
LOR                 0
CGPA                0
Research            0
ChanceofAdmit       0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
# your code here
db.set_index('SerialNo.')

,GREScore,TOEFLScore,UniversityRating,SOP,LOR,CGPA,Research,ChanceofAdmit
SerialNo.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [7]:
# your code here
db.set_index('GREScore','CGPA')

,SerialNo.,TOEFLScore,UniversityRating,SOP,LOR,CGPA,Research,ChanceofAdmit
GREScore,,,,,,,,
337,1,118,4,4.5,4.5,9.65,1,0.92
316,2,104,3,3.0,3.5,8.00,1,0.72
322,3,110,3,3.5,2.5,8.67,1,0.80
314,4,103,2,2.0,3.0,8.21,0,0.65
330,5,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
324,381,110,3,3.5,3.5,9.04,1,0.82
325,382,107,3,3.0,3.5,9.11,1,0.84
330,383,116,4,5.0,4.5,9.45,1,0.91


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [8]:
# your code here
# db.reset_index(drop=True)
db.reset_index()
db

,SerialNo.,GREScore,TOEFLScore,UniversityRating,SOP,LOR,CGPA,Research,ChanceofAdmit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [9]:
# your code here

db[db['CGPA']>9].head()

,SerialNo.,GREScore,TOEFLScore,UniversityRating,SOP,LOR,CGPA,Research,ChanceofAdmit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
4,5,330,115,5,4.5,3.0,9.34,1,0.90
10,11,328,112,4,4.0,4.5,9.10,1,0.78
19,20,328,116,5,5.0,5.0,9.50,1,0.94
20,21,334,119,5,5.0,4.5,9.70,1,0.95


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [10]:
# your code here

db['greater 9 & less 3.5']= (db['CGPA']>9) & (db['SOP'] < 3.5)
db['mean'] =db['ChanceofAdmit'].mean()

db[['greater 9 & less 3.5','mean']].head()


,greater 9 & less 3.5,mean
0,False,0.723922
1,False,0.723922
2,False,0.723922
3,False,0.723922
4,False,0.723922


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [11]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    # your code here
    result = (col - col.mean())/col.std()
    return result   

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [12]:
# your code here
db['CGPA_std'] =standardize(db['CGPA'])
db['GRE_std'] = standardize(db['GREScore'])
db['LOR_std'] = standardize(db['LOR'])
db

,SerialNo.,GREScore,TOEFLScore,UniversityRating,SOP,LOR,CGPA,Research,ChanceofAdmit,greater 9 & less 3.5,mean,CGPA_std,GRE_std,LOR_std
0,1,337,118,4,4.5,4.5,9.65,1,0.92,False,0.723922,1.747900,1.753381,1.191647
1,2,316,104,3,3.0,3.5,8.00,1,0.72,False,0.723922,-0.991211,-0.063367,0.076740
2,3,322,110,3,3.5,2.5,8.67,1,0.80,False,0.723922,0.121034,0.455704,-1.038166
3,4,314,103,2,2.0,3.0,8.21,0,0.65,False,0.723922,-0.642597,-0.236391,-0.480713
4,5,330,115,5,4.5,3.0,9.34,1,0.90,False,0.723922,1.233279,1.147798,-0.480713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,False,0.723922,0.735259,0.628727,0.076740
381,382,325,107,3,3.0,3.5,9.11,1,0.84,True,0.723922,0.851463,0.715239,0.076740
382,383,330,116,4,5.0,4.5,9.45,1,0.91,False,0.723922,1.415886,1.147798,1.191647
383,384,312,103,3,3.5,4.0,8.78,0,0.67,False,0.723922,0.303641,-0.409414,0.634193


We will generate the decision choice at random using the code below. Please run the cell.

In [13]:
# Libraries
from random import choices

In [14]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=db.shape[0])



Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [15]:
# your code here

deciding_column = db.lookup(db.index,decision_choice)
db['deciding_column'] =deciding_column


db['deciding_column']

<ipython-input-15-a44a2baf8b95>:3: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  deciding_column = db.lookup(db.index,decision_choice)


0      1.747900
1      0.076740
2      0.455704
3     -0.480713
4     -0.480713
         ...   
380    0.628727
381    0.715239
382    1.191647
383   -0.409414
384    0.634193
Name: deciding_column, Length: 385, dtype: float64

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [16]:
# your code here

db['decision']=[1 if col > 0.8 else 0 for col in db['deciding_column']]
db['decision'].dtype

dtype('int64')

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [87]:
# your code here

db['decision'].sum()

89

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [90]:
# your code here
db.columns = [letter.strip('.').replace(' ', '_').lower() for letter in db.columns]
db

,serialno,grescore,toeflscore,universityrating,sop,lor,cgpa,research,chanceofadmit,greater_9_&_less_3.5,mean,cgpa_std,gre_std,lor_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,False,0.723922,1.747900,1.753381,1.191647,1.747900,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,False,0.723922,-0.991211,-0.063367,0.076740,0.076740,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,False,0.723922,0.121034,0.455704,-1.038166,0.455704,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,False,0.723922,-0.642597,-0.236391,-0.480713,-0.480713,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,False,0.723922,1.233279,1.147798,-0.480713,-0.480713,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,False,0.723922,0.735259,0.628727,0.076740,0.628727,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,True,0.723922,0.851463,0.715239,0.076740,0.715239,0
382,383,330,116,4,5.0,4.5,9.45,1,0.91,False,0.723922,1.415886,1.147798,1.191647,1.191647,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,False,0.723922,0.303641,-0.409414,0.634193,-0.409414,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [93]:
# your code here
db['adjusted_gre'] = pd.cut(db[db['universityrating'] >= 4]['grescore'].add(10), bins= 4)
db

,serialno,grescore,toeflscore,universityrating,sop,lor,cgpa,research,chanceofadmit,greater_9_&_less_3.5,mean,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,1,337,118,4,4.5,4.5,9.65,1,0.92,False,0.723922,1.747900,1.753381,1.191647,1.747900,1,"(337.5, 350.0]"
1,2,316,104,3,3.0,3.5,8.00,1,0.72,False,0.723922,-0.991211,-0.063367,0.076740,0.076740,0,NaN
2,3,322,110,3,3.5,2.5,8.67,1,0.80,False,0.723922,0.121034,0.455704,-1.038166,0.455704,0,NaN
3,4,314,103,2,2.0,3.0,8.21,0,0.65,False,0.723922,-0.642597,-0.236391,-0.480713,-0.480713,0,NaN
4,5,330,115,5,4.5,3.0,9.34,1,0.90,False,0.723922,1.233279,1.147798,-0.480713,-0.480713,0,"(337.5, 350.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,False,0.723922,0.735259,0.628727,0.076740,0.628727,0,NaN
381,382,325,107,3,3.0,3.5,9.11,1,0.84,True,0.723922,0.851463,0.715239,0.076740,0.715239,0,NaN
382,383,330,116,4,5.0,4.5,9.45,1,0.91,False,0.723922,1.415886,1.147798,1.191647,1.191647,1,"(337.5, 350.0]"
383,384,312,103,3,3.5,4.0,8.78,0,0.67,False,0.723922,0.303641,-0.409414,0.634193,-0.409414,0,NaN
